**<center><font size=5>SETI Breakthrough Listen - E.T. Signal Search 👽</font></center>**
<center><font size=4>By Ben Shaver and Coral Hayoun</font></center>

----
# Abstract

##### **"Are we alone in the Universe?"**

It's one of the most profound human questions.
As technology improves, we're finding new and more powerful ways to seek for answers.

The Breakthrough Listen team at Berkeley University, scans signals from millions of stars in the universe using world's most powerful telescopes, trying
to answer this question. Unfortunately, finding extraterrestial signals is not that easy - by the time, humans have built enormous numbers of radio devices that can be detected as signals as well and distract researchers.

In order to separate outer space signals from RFI (radio frequency interference), The Breakthrough Listen team are using two different filters:
1. They intersperses scans of target stars that apprears in more than one regions of the sky - because these scans probably aren't coming from the direction of the target star.
2. They discards signals that don't change their frequency - because it means that they are probably nearby the telescope.

These two methods are quite effective, but they can be improved. The process today misses interesting signals. particularly those with complex time or frequency structure, and those in regions of the spectrum with lots of interference.

<br></br>
In this competition, we will use our machine learning skills to accurately identify the presence of simulated extraterrestial signals in these scans.

Because there are no confirmed examples of alien signals to use to train machine learning algorithms, the team included some simulated signals (that they call “**needles**”) in the haystack of data from the telescopes. They have identified some of the hidden needles so that we can train our model to find more.

<br></br>
<center><img src="https://i.pinimg.com/originals/89/e6/91/89e6912b1225c43ed18b7c2b31069f77.jpg" width="600"></center>

(from [SETI Breakthrough Listen - Kaggle competition](https://www.kaggle.com/competitions/seti-breakthrough-listen)) <br></br>

----

##### **Table of Contents**
- <a href='#1'>1. Initial Settings | ⚙️</a>
- <a href='#2'>2. Exploratory Data Analysis | 🔦</a>
  - <a href='#2_1'>2.1. Data Loading</a>
  - <a href='#2_2'>2.2. Data Visualizing</a>
  - <a href='#2_3'>2.3. Data Statistics</a>
    - <a href='#2_3_1'>2.3.1 Needles Distribution</a>
    - <a href='#2_3_2'>2.3.2 Basic Statistical Features</a>
  - <a href='#2_4'>2.4. Noise Analyzing</a>
    - <a href='#2_4_1'>2.4.1 Gaussian Noise</a>
    - <a href='#2_4_2'>2.4.2 Frequency Domain Analysis</a>
    - <a href='#2_4_3'>2.4.3 Wavelet Transform Analysis</a>
  - <a href='#2_5'>2.5. Clustering</a>
    - <a href='#2_5_1'>2.5.1 Data Respresentation</a>
      - <a href='#2_5_1_1'>2.5.1.1. Vertical Detail Coefficients (cV)</a>
      - <a href='#2_5_1_2'>2.5.1.2. SIFT</a>
      - <a href='#2_5_1_3'>2.5.1.3. HOG</a>
      - <a href='#2_5_1_4'>2.5.1.4. LBP</a>
    - <a href='#2_5_2'>2.5.2 Dimensionality Reduction - PCA</a>
    - <a href='#2_5_3'>2.5.3 K-means</a>
- <a href='#3'>3. Data Preprocessing | 🍽️</a>
  - <a href='#3_1'>3.1. Data Cleaning</a>
  - <a href='#3_2'>3.2. Data Augmentation</a>
  - <a href='#3_3'>2.3. Data Adjustments</a>
- <a href='#4'>4. Data Splitting | 🔪</a>
- <a href='#5'>5. Train A Simple Model | 🐇</a>
  - <a href='#5_1'>5.1. Model Building</a>
  - <a href='#5_2'>5.2. Preprocessing Pipeline</a>
  - <a href='#5_3'>5.3. Training Function</a>
  - <a href='#5_4'>5.4. Predictions</a>
  - <a href='#5_5'>5.5. Final Results</a>
- <a href='#6'>6. Train An Improved Simple Model | 🦍</a>
- <a href='#7'>7. Train A Deep Learning Model | 🤖</a>
  - <a href='#7_1'>7.1. Model Building</a>
  - <a href='#7_2'>7.2. Preprocessing Pipeline</a>
  - <a href='#7_3'>7.3. Training Function</a>
  - <a href='#7_4'>7.4. Predictions</a>
  - <a href='#7_5'>7.5. Final Results</a>
- <a href='#8'>8. Conclusions | 💡</a>


  ----

# <a id='1'>1. Initial Settings | ⚙️</a>

##### 1. Install Libraries

In [ ]:
%pip install -q pywavelets

##### 2. Import Libraries

In [ ]:
# For operating system and file system
import os
import time
import logging

# For basic functionallity
import math

# For data manipulation and analysis
import numpy as np
import pandas as pd
import pywt

# For Computer Vision Library
import cv2
from skimage import exposure
from skimage.feature import hog
from skimage.feature import local_binary_pattern

# For Deep Learning
import tensorflow as tf
import efficientnet.tfkeras as efficientnet
from tensorflow.keras import layers
from tqdm.keras import TqdmCallback

# For Machine Learning
import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.cluster import MiniBatchKMeans
from sklearn.utils import shuffle

# For data manipulation
from skimage.transform import resize
from skimage.restoration import estimate_sigma

# For data visualization
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns

# For model persistence
import joblib

# For warning filtering
import warnings

In [ ]:
# Sometimes it can be necessary to re-run this command for plots to show automatically
%matplotlib inline

##### 3. Libraries Version

In [ ]:
print('np version:',np.__version__)
print('pd version:',pd.__version__)
print('tf version:',tf.__version__)
print('sklearn version:',sklearn.__version__)

##### 4. Configuration

In [ ]:
class CFG:
    # ------------------------------------#
    # Basic
    # ------------------------------------#
    debug = True
    runOnKaggle = False
    epochs = 1
    train_batch_size = 64
    validation_batch_size = 20
    data_augmentation_size = 5000
    test_batch_size = 64
    seed = 42
    k_fold = 4

    # ------------------------------------#
    # Paths
    # ------------------------------------#
    if runOnKaggle:
        base_folder_path = '/kaggle/input/seti-breakthrough-listen'
        tmp_storage_path = '/kaggle/tmp'
        workdir_folder_path = '/kaggle/working'
        logging_file = '/kaggle/logging.txt'
    else:
        base_folder_path = os.path.abspath('../../../Downloads/SETI')
        tmp_storage_path = 'tmp'
        workdir_folder_path = 'working'
        logging_file = 'logging.txt'

    train_folder_path = f'{base_folder_path}/train'
    test_folder_path = f'{base_folder_path}/train'

# ------------------------------------#
# On Debug
# ------------------------------------#
if CFG.debug:
    CFG.epochs = 1
    CFG.train_batch_size = 50
    CFG.validation_batch_size = 25
    CFG.test_batch_size = 50
    CFG.data_augmentation_size = 500

##### 5. Utils Methods

In [ ]:
def get_batched_data (data, batch_size, index):
    return data[index * batch_size : (index + 1) * batch_size]

def split_data(data, k):
    np.random.shuffle(data)
    return np.array_split(data, k)

def plot_cadence_snippet(cadence_snippet, figsize=(8, 5)):
  figure = plt.figure(figsize=figsize)
  
  for i in range(6):
    data = cadence_snippet[i]
    channel_text = 'on' if i % 2 == 0 else 'off'
    channel_box = dict(facecolor='white', edgecolor='none', boxstyle='square,pad=0.3')
    
    plt.subplot(6, 1, i + 1)
    plt.imshow(data.astype('float16'), aspect='auto', cmap='gray')
    plt.text(10, 90, channel_text, color='black', fontsize=8, va='center', ha='center',bbox=channel_box)

  figure.text(0.5, 0.04, 'Frequency', ha='center', fontsize=8)
  figure.text(0.04, 0.5, 'Time', va='center', rotation='vertical', fontsize=8)
  plt.show()

def plot_cadence_snippet_by_axes(cadence_snippet, axes):
    for i in range(6):
        channel_text = 'on' if i % 2 == 0 else 'off'
        channel_box = dict(facecolor='white', edgecolor='none', boxstyle='square,pad=0.3')
        
        ax = axes[i]
        ax.imshow(cadence_snippet[i].astype('float16'), aspect='auto', cmap='gray') 
        ax.text(10, 90, channel_text, color='black', fontsize=8, va='center', ha='center', bbox=channel_box)
        ax.axis('off') 

def plot_multiple_cadence_snippets(cadences_snippet, figsize=(15, 10)):
    rows_number = (len(cadences_snippet) + 2) // 3 

    figure = plt.figure(figsize=figsize)
    grid = figure.add_gridspec(rows_number * 6 + (rows_number - 1), 3 * 6 + 2) 

    for index, cadence_snippet in enumerate(cadences_snippet):
        row_start = (index // 3) * 7  # 6 rows for plots + 1 row for margin
        column_start = (index % 3) * 7   # 6 columns for plots + 1 column for margin
        
        axes = [figure.add_subplot(grid[row_start + i, column_start:column_start + 6]) for i in range(6)]
        plot_cadence_snippet_by_axes(cadence_snippet, axes)

    plt.show()

def plot_on_channels_cadence_snippet(cadence_snippet):
  figure = plt.figure(figsize=(8, 4))
  
  for i in range(3):
    plt.subplot(6, 1, i + 1)
    plt.imshow(cadence_snippet[i], aspect='auto')

  figure.text(0.5, 0.04, 'Frequency', ha='center', fontsize=8)
  figure.text(0.04, 0.5, 'Time', va='center', rotation='vertical', fontsize=8)
  plt.show()

def plot_merged_cadence_snippet(file, title = None):
  plt.figure(figsize=(8, 5))

  if title is not None: 
    plt.title(title) 
    
  plt.imshow(file, aspect='auto')

def get_samples_id(df, target_0_amount, target_1_amount):
    id_target_0 = df[df['target'] == 0].head(target_0_amount)['id'].tolist()
    id_target_1 = df[df['target'] == 1].head(target_1_amount)['id'].tolist()
    return id_target_0 + id_target_1

def get_samples_by_threshold(df, feature, low_threshold, high_threshold):
    random_state = None
    low_threshold = np.percentile(df[feature], low_threshold)
    high_threshold = np.percentile(df[feature], high_threshold)
    median_threshold = np.median(df[feature])
    median_low_threshold = median_threshold - 0.1 * median_threshold
    median_high_threshold = median_threshold + 0.1 * median_threshold

    lowest = shuffle(df[df[feature] < low_threshold], random_state=random_state)
    highest = shuffle(df[df[feature] > high_threshold], random_state=random_state)
    median = shuffle(df[(df[feature] >= median_low_threshold) & (df[feature] <= median_high_threshold)], random_state=random_state)

    return lowest, median, highest

def write_predictions_to_file(prediction_data, testing_set_labels, file_name):
  results = testing_set_labels.assign(target=prediction_data)
  selected_column = ['id', 'target']
  final_results = results[selected_column]
  final_results.to_csv(file_name, index=False)

def get_batch_number(index, batch_size):
  return math.floor(index / batch_size)

def get_index_in_batch(index, batch_size):
  return index - (get_batch_number(index, batch_size) * batch_size)

def get_data_set_sample(data_set, n, positive_ratio):
    positive_values_samples = min(math.floor(n * positive_ratio),len(data_set[data_set.target == 1]))
    positive_samples = data_set[data_set.target == 1].sample(n=positive_values_samples, random_state = CFG.seed).reset_index(drop=True)
    complementary_data_set_samples = data_set[data_set.target == 0].sample(n - positive_values_samples, random_state = CFG.seed).reset_index(drop=True)

    return pd.concat([positive_samples, complementary_data_set_samples]).sample(frac = 1).reset_index(drop=True)

def log(message, shouldPrint=False):
  if shouldPrint:
    print(message)
  logging.info(message)

def save_model(model, name):
  joblib.dump(model, f'{CFG.workdir_folder_path}/{name}.pkl')
  
def load_model(name):
  return joblib.load(f'{CFG.workdir_folder_path}/{name}.pkl')

def get_duration(start_time):
  return (time.time() - start_time) / 60

##### 6. Initial Methods

In [ ]:
# Create tmp Directory
if not os.path.exists(CFG.tmp_storage_path):
    os.makedirs(CFG.tmp_storage_path)

# Create working Directory
if not os.path.exists(CFG.workdir_folder_path):
    os.makedirs(CFG.workdir_folder_path)
    
# Create logging file
if not os.path.exists(CFG.logging_file):
    with open(CFG.logging_file, 'w'):
        pass

# Setup logger
logging.basicConfig(
    level=logging.INFO,  
    format='%(asctime)s - %(message)s',  
    filename=CFG.logging_file,  
    filemode='a'  
  )

# Remove unnecessary warnings
warnings.filterwarnings('ignore')

----

# <a id='2'>2. Exploratory Data Analysis | 🔦</a>


With our libraries and configurations set up, we can begin the exploratory data analysis (or EDA).

The **goal of EDA** is to understand the data underlying structure and patterns, identify important variables, detect outliers and anomalies and formulate hypotheses for further investigation. 

This process includes examining the distribution of the data, identifying patterns and trends, applying dimensionality reduction and clustering techniques, and visualizing the data to gain insights.

### <a id='2_1'>2.1. Data Loading</a>

Even though our task sounds like a *signal* detection problem, we are actually provided with radio spectrograms *images* - a visual representation of the spectrum frequencies of a signal as it varies with time. <br></br>
Our full dataset contains:
- a `train_labels.csv` file, which has our labels indicating the presence of an alien signal (referred to as needles) for each of the spectograms
- `train` folder contains `.npy` float16 files with the spectrograms stored as arrays
- Similarly, `test` folder also contains `.npy` float16 files with the test spectrograms stored as arrays <br></br>

Because the given testing set is unlabeled, it will be hard to work with it - when we will train our model and make predictions, we will get by Kaggle the accuracy percentages of the predictions only, without the ability to detect the model mistakes or use some more subtle measures. Therefore, we will split our labeled training set using `train_test_split` method into two - 80% of it will keep being our training set, and 20% will become our labeled testing set.

In [ ]:
training_set_labels = pd.read_csv(f'{CFG.base_folder_path}/train_labels.csv')

if CFG.debug:
    samples_amount = 2000
    positive_samples_ratio = 0.3
    training_set_labels = get_data_set_sample(training_set_labels, samples_amount, positive_samples_ratio)

training_set, testing_set = train_test_split(training_set_labels, test_size=0.2, random_state=CFG.seed)

# Keep the splitted labels into train and test csv files
training_set.to_csv('training_set_labels.csv', index=False)
testing_set.to_csv('testing_set_labels.csv', index=False)

training_set_labels.info()

Looks like we have a total of 60,000 data samples, which means we have 48,000 training set samples and 12,000 testing set samples.<br></br>
As we can see, each data sample has an id and stored as a `.npy` file with the path of 'train/{image_id_prefix}/{image_id}.npy'

In [ ]:
def get_file_path(image_id, folder_path):
  return '{}\{}\{}.npy'.format(folder_path, image_id[0], image_id)

training_set['file_path'] = training_set['id'].apply(lambda id: get_file_path(image_id=id, folder_path=CFG.train_folder_path))
testing_set['file_path'] = testing_set['id'].apply(lambda id: get_file_path(image_id=id, folder_path=CFG.test_folder_path))

training_set.head()

After understanding the way our data being stored, we should make sure that our stored data is valid and consistent:
- Each record in the csv files should actually has a valid (not nullish) id and label
- There are no missing files, which means all the labeled records in the csv files actually has a matched existing file
- There are no extra files, which means each existing file actually has a labeled record in our csv files

In [ ]:
required_columns = ['id', 'target']
cleaned_training_set_labels = training_set_labels.dropna(subset=required_columns)

if cleaned_training_set_labels.shape[0] < training_set_labels.shape[0]:
    print('Some records has invalid values')
else:
    print('All records are valid')

In [ ]:
csv_ids = set(training_set_labels['id'].astype(str))
file_ids = set()

for root, dirs, files in os.walk(CFG.train_folder_path):
    for file_name in files:
        # Remove file extension
        file_id = os.path.splitext(file_name)[0]
        file_ids.add(file_id)

missing_files = csv_ids - file_ids
extra_files = file_ids - csv_ids

if missing_files:
    print(f'Files referenced in CSV but missing in train directory: {missing_files}')
else:
    print('All files referenced in CSV are present in the train directory.')

if extra_files:
    print(f'Files in train directory but not referenced in CSV: {extra_files}')
else:
    print('All files in the train directory are referenced in the CSV.')

Great. Looks like our stored dataset is consistent and valid to use, now we can delve into its real exploration:

### <a id='2_2'>2.2. Data Visualizing</a>

Every training set sample is called **cadence snippet**, which looks like this:

In [ ]:
cadence_snippet = np.load(get_file_path('00776881dd80050', CFG.train_folder_path))
print(f'My shape is: {cadence_snippet.shape}')
plot_cadence_snippet(cadence_snippet)

Let's break this **cadence snippet** down:

**Firstly**, the individual plots signify a **cadence**, and the combination of the 6 plots is called a **cadence snippet**

**Secondly**, as we can see, the spectrogram has a shape of `(6, 273, 256)` - which stands for a represantation of 6 spectrograms in 2D.
The reason for the 6 spectrograms, as we already mentioned in the Abstract section, is that in order to deal with human-generated signals (like radion stations or wifi routers), Breakthrough Listen team are looking for **signals that appear to be coming from particular positions on the sky**.

Typically they do this by alternating observations of a primary target star with observations of three nearby stars:
- 5 minutes on star “A” || *on-channel*
- then 5 minutes on star “B” || *off-channel*
- then back to star “A” for 5 minutes || *on-channel*
- then “C” || *off-channel*
- then back to “A” || *on-channel*
- then finishing with 5 minutes on star “D” || *off-channel*

One set of six observations (ABACAD) is referred to as our **cadence snippet**.

**Thirdly**, The curves we see in the signal's lines, are because the relative motion of the Earth and the star imparts a `Doppler drift`, causing the frequency to change over time. 


After understanding what cadence snippet is, let's take a look on some more cadence snippets in the training set:

In [ ]:
print('Here is a non-alien signals example:')
plot_cadence_snippet(np.load(get_file_path('0000799a2b2c42d', CFG.train_folder_path)))

print('\nHere is an easy alien signals example:')
plot_cadence_snippet(np.load(get_file_path('d618b77bb0909c2', CFG.train_folder_path)))

print('\nHere is a hard alien signals example:')
plot_cadence_snippet(np.load(get_file_path('fffbb1c9c3d6c31', CFG.train_folder_path)))

Here we came across with our **1st challange: hidden needles**. As we can see, some of these needles should be easy to detect, but others are hidden in noisy regions of the spectrum and will be harder. 


<div style="border: 0.5px solid; padding: 10px; background-color: #FFFACD; border-radius: 5px; color: #000; font-size:14px; line-height: 1.7em; display: flex; flex-direction: row;">
  <div style="width: 24px">💡</div> 
  <div style="display: flex; flex-direction: column"> 
  <div style="font-weight:500">How useful are the off-channels? </div>
  <div>
  As we mentioned above, a signal that only present when looking at the on-channels might be an alien signal - but if it also shows up in an off-channel we can assume it's just a boring terrestrial signal. How useful are the off-channels is a serious question for the contest, and potentially for the SETI project itself. Theoretically, the scenario that off-channels will rule out the possibility that a signal is a neelde can be very rare - so, in this case maybe its might not worth the usage of it. In the further parts of this project, we will face this question and answer it.
  </div>
  </div>
</div>

### <a id='2_3'>2.3. Data Statistics</a>

#### <a id='2_3_1'>2.3.1 Needles Distribution</a>

Let's create a barplot of the label frequencies in order to understand how rare is having a "needle" occurance:

In [ ]:
train_needles = len(training_set[training_set.target==1])
test_needles = len(testing_set[testing_set.target==1])

print(f"There are {train_needles} 'needles' in the training set, which is {(train_needles / len(training_set) * 100)} % of the set. \n")
print(f"There are {test_needles} 'needles' in the testing set, which is {(test_needles / len(testing_set) * 100)} % of the set. \n")

sns.countplot(data=training_set, x='target')

By looking at this plot, we can see that our **2nd challange** is a **very imbalanced dataset** - most of the examples are non-alien. If we will use this dataset for our predictive models without any adjustments, we might get a lot of misclassifications.

> **Imbalance** means that the number of data points available for different classes is different

#### <a id='2_3_2'>2.3.2 Basic Statistical Features</a>

Now, let's take a look on some basic spectrogram's statistical features in order to look for outliers or hopefully find a clear difference between the alien and the non-alien ones:
- **Mean**: The avarage pixels value in an image, provides a measure of the *overall brightness* of the image.
- **Standard Deviation**: A measure of the dispersion of pixel values around the mean, indicating the variability in pixel values. Higher standard deviation values suggest greater contrast in the image.
- **Min, Max**: The minimum and the maximum pixels value in an image, which will probably be 0 and 255 (black and white)


<div style="border: 0.5px solid; padding: 10px; background-color: #df5e55c2; border-radius: 5px; color: #000; font-size:14px; line-height: 1.7em; display: flex; flex-direction: row;">
  <div style="width: 24px">📢</div> 
  <div>
  From now on, we will explore the training set only, with the assumption that our testing set should be pretty similar statistically.
  </div>
</div>

In [ ]:
def extract_features(spectrograms):
    return {
        'mean': np.mean(spectrograms),
        'std': np.std(spectrograms),
        'max': np.max(spectrograms),
        'min': np.min(spectrograms)
    }

# Main
features_list = []

for _, row in training_set.iterrows():
    file_id = row['id']
    file_path = get_file_path(file_id, CFG.train_folder_path)
    file = np.load(file_path)
    spectrograms = np.vstack(file)

    features = extract_features(spectrograms)
    features['id'] = file_id
    features['target'] = row['target']

    features_list.append(features)

features_df = pd.DataFrame(features_list)

After collecting these features, let's analyze them:

In [ ]:
def plot_statistical_features(features):
    _, axes = plt.subplots(math.ceil(len(features) / 2), 2, figsize=(10, 7))
    axes = axes.flatten()

    for i, feature in enumerate(features):
        # sns.histplot needs convertion from float16 to float32
        features_df[feature] = features_df[feature].astype('float32')
        sns.histplot(features_df[features_df['target'] == 0][feature], bins=30, kde=True, color='blue', label='0', ax=axes[i])
        sns.histplot(features_df[features_df['target'] == 1][feature], bins=30, kde=True, color='orange', label='1', ax=axes[i])
        
        # Set title and labels
        axes[i].set_title(f'Distribution of {feature} by Target')
        axes[i].set_xlabel(feature)
        axes[i].set_ylabel('Count')
        axes[i].legend()

    plt.tight_layout()
    plt.show()

# Main
plot_statistical_features(['mean', 'std', 'max', 'min'])

non_inf_std_count = np.isfinite(features_df['std']).sum()
print(f'Number of std values that are not inf: {non_inf_std_count}')

Before we will conclude anything from the plots, we should handle the standard deviation (`std`) value, which appears to be `inf` for both the non-alien and alien targets. There can be a few reasons for that - the first one, is **overflow issues** during intermediate calculations, which also *match the overflow warning we can see above*. Another reason can be extreme values in the dataset (such as `inf` or `NaN`) that can significantly inflate `std`.

We will try to fix the `std` calculation value using few data tweaks:
- We will cast our data from `float16` to `float32`, that has higher precision to avoid overflow.
- We will look for `inf` or `NaN` pixels values and replace them with finite values

In [ ]:
def fix_std_value(cadence_snippet):
    cadence_snippet = cadence_snippet.astype(np.float32)
    finite_max = np.max(cadence_snippet[np.isfinite(cadence_snippet)])
    finite_min = np.min(cadence_snippet[np.isfinite(cadence_snippet)])
    
    return np.nan_to_num(cadence_snippet, nan=0.0, posinf=finite_max, neginf=finite_min)

In [ ]:
features_list = []

for _, row in training_set.iterrows():
    file_id = row['id']
    file_path = get_file_path(file_id, CFG.train_folder_path)
    file = np.load(file_path)
    cadence_snippet = np.vstack(file)

    features = {
        'inf_values': np.sum(np.isinf(cadence_snippet)),
        'nan_values': np.sum(np.isnan(cadence_snippet))
    }

    cadence_snippet = fix_std_value(cadence_snippet)

    features.update(extract_features(cadence_snippet))
    features['id'] = file_id
    features['target'] = row['target']

    features_list.append(features)

features_df = pd.DataFrame(features_list)

In [ ]:
plot_statistical_features(['inf_values', 'nan_values', 'mean', 'std', 'max', 'min'])

non_alien_inf_values = np.sum(np.isinf(features_df[features_df['target'] == 0]['inf_values']))
non_alien_nan_values = np.sum(np.isinf(features_df[features_df['target'] == 0]['nan_values']))
alien_inf_values = np.sum(np.isinf(features_df[features_df['target'] == 1]['inf_values']))
alien_nan_values = np.sum(np.isinf(features_df[features_df['target'] == 1]['nan_values']))

print(f'Number of inf values in target 0: {non_alien_inf_values}, in target 1: {alien_inf_values}')
print(f'Number of nan values in target 0: {non_alien_nan_values}, in target 1: {alien_nan_values}')

Great! Looks like our `std` is fine now. As we can see there are no `inf` of `NaN` pixels values, and there is no overflow warning anymore. We can infer that he `std` fixed by the `float32` casting. There are few more things we can infer as well:
- The aliens and non-aliens samples have a pretty similar graphs trends in all of our checked statistical features. Hence, non of these features can potentially serve as discriminative factor.
- The `std` is pretty small - which means the variance of our each cadence snippet is very small. **small data variance** will consider as our **3rd challange** for a few reasons, like - the challange of preserving essential features while preforming dimensionality reduction or resizing.


Now, we will try to look at the outliers:

In [ ]:
def display_outliers(lowest_df, highest_df, target_0_amount, target_1_amount, feature):
    figsize=(14, 4)
    lowest_ids = get_samples_id(lowest_df, target_0_amount, target_1_amount)
    print(f'Lowest {feature} candece snippets:')
    [print(lowest_df.loc[lowest_df['id'] == id, feature].values[0]) for id in lowest_ids]
    
    cadence_snippets = [np.load(get_file_path(id, CFG.train_folder_path)) for id in lowest_ids]
    plot_multiple_cadence_snippets(cadence_snippets, figsize=figsize)

    highest_ids = get_samples_id(highest_df, target_0_amount, target_1_amount)
    print(f'Highest {feature} candece snippets:')
    [print(highest_df.loc[highest_df['id'] == id, feature].values[0]) for id in highest_ids]

    cadence_snippets = [np.load(get_file_path(id, CFG.train_folder_path)) for id in highest_ids]
    plot_multiple_cadence_snippets(cadence_snippets, figsize=figsize)

# Main
lowest, medium, highest = get_samples_by_threshold(features_df, 'std', 5, 95)
display_outliers(lowest, highest, 2, 1, 'std')

lowest, medium, highest = get_samples_by_threshold(features_df, 'mean', 5, 95)
display_outliers(lowest, highest, 2, 1, 'mean')

As we can see, the `std` and `mean` values has a very narrow range of values. We can also see, by looking at the images - that the outliers of these two features are just fine to work with, and there is **no outliers that needs to be removed at this point**.

### <a id='2_4'>2.4 Noise Analyzing</a>
As we saw in the Data Visualizing section and in the Standard Devition calculation, our data is not varying much and it also full of noises. Noise Analyzing will help us understand better our dataset for few reasons:
- We can use Noise Analysis to determine if this lack of variation is due to actual data properties or if it's caused by noise. 
- We can try find noise indicators which are also discriminative factors between alien and non-alien signal
- We will use this analyzing for removing some of the noises in the next sections.

#### <a id='2_4_1'>2.4.1 Gaussian Noise</a>
`Gaussian Noise`, also known as normal noise, is a fundamental concept in signal processing and statistics. It is called *Gaussian* because its probability density function (*PDF*) follows a Gaussian (normal) distribution. By estimating the standard deviation of the Gaussian noise in the images, we can gain insights into the level of noise present and its impact on the image quality.

Let's analyze the Gaussian noise std using `estimate_sigma ` function, which based on **Wavelet Decomposition** to estimate the noise level in an image:
> **Wavelet Decomposition** - Wavelets are mathematical functions that can be used to divide a function or a continuous-time signal into different frequency components, this helps isolate the high-frequency noise from the low-frequency signal.

In [ ]:
gaussian_noise_list = []

for _, row in training_set.iterrows():
    file_id = row['id']
    file_path = get_file_path(file_id, CFG.train_folder_path)
    file = np.load(file_path)

    spectrograms = np.vstack(file)
    spectrograms = spectrograms.astype(np.float32)

    gaussian_noise = {
        'noise': estimate_sigma(spectrograms, channel_axis=None, average_sigmas=True)
    }
    gaussian_noise['id'] = file_id
    gaussian_noise['target'] = row['target']

    gaussian_noise_list.append(gaussian_noise)

gaussian_noise_df = pd.DataFrame(gaussian_noise_list)

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)

# Plot distribution of Gaussian Noise by target
sns.histplot(gaussian_noise_df[gaussian_noise_df['target'] == 0]['noise'], bins=30, kde=True, color='blue', label='Target 0')
sns.histplot(gaussian_noise_df[gaussian_noise_df['target'] == 1]['noise'], bins=30, kde=True, color='orange', label='Target 1')
plt.title('Distribution of Gaussian Noise by Target')
plt.xlabel('Estimated Noise Level')
plt.ylabel('Count')
plt.legend()

plt.tight_layout()
plt.show()

Looks like most of our images has a low rate of Gaussian Noise, which generally indicates that most of the data is relatively "clean". In order to understand what clean means in our type of data, let's take a look over some outliers and some images that are just in the median Gaussian distribution.

We will plot 3 non-alien signals and then 3 alien signals for: lowest noise images, median noise images and highest noise cadence snippets:

In [ ]:
non_aliens_amount = training_set['target'].value_counts().get(0,0)
aliens_amount = training_set['target'].value_counts().get(1,0)

def display_noise_images(noise_images_df, title):
    target_counts = noise_images_df['target'].value_counts()
    non_aliens_ratio = (target_counts.get(0, 0) / non_aliens_amount) * 100
    aliens_ratio = (target_counts.get(1, 0) / aliens_amount) * 100
    images_id = get_samples_id(noise_images_df, 3, 3)

    print(title)
    print(f'Target 0: {non_aliens_ratio}%, Target 1: {aliens_ratio}%')

    cadences_snippet = [np.load(get_file_path(id, CFG.train_folder_path)) for id in images_id]
    plot_multiple_cadence_snippets(cadences_snippet, figsize=(18, 12))

# Main
lowest_noise_images, median_noise_images, highest_noise_images = get_samples_by_threshold(gaussian_noise_df, 'noise', 10, 90)

display_noise_images(lowest_noise_images, '\nLowest Gaussian noise images:')
display_noise_images(median_noise_images, '\nMedian Gaussian noise images:')
display_noise_images(highest_noise_images, '\nHighest Gaussian noise images:')

Interesting images! We knew that the less noise we have - the cleaner the image is. But, we can now sense and understand how the noise affect image clearity.

As we can see, The lowest noise images are very clear and clean, and some of its needels are pretty noticable, but yet - some of them doesnt. The median noise images are less clear and so does the needels. The highest noise images has a very bad quality and it like the needels finding is the hardest, 

<div style="border: 0.5px solid; padding: 10px; background-color: #df5e55c2; border-radius: 5px; color: #000; font-size:14px; line-height: 1.7em; display: flex; flex-direction: row;">
  <div style="width: 24px">📢</div> 
  <div>
  If we will see in the training section that the prediction of this type of images are less accurate, we will consider to remove them from our dataset.
  </div>
</div>


Although it looks like the mean noise of our images is pretty low, there are still a lot of static frequencies that create "noise" and makes the needels harder to find. Let's explore them:

#### <a id='2_4_2'>2.4.2 Frequency Domain Analysis</a>

`Frequency Domain Analysis` is a powerful technique used in image processing to examine the frequency components of an image. This analysis transforms an image from the spatial domain (where pixel values are analyzed) to the frequency domain, where the image is represented by its frequency components.

The core of `Frequency Domain Analysis` is the `Fourier Transform`, which decomposes an image into its constituent frequencies. So, we need to start with understanding `Fourier Transform` meaning:

In the picture below, we can see the **Time Domain** representation of a audio signal, which shows the "loudness" (amplitude) of sounds wave changing with time. To better understand an audio signal, it is necessary to transform it into the **Frequency Domain** representation. This representation of a signal tells us what different frequencies are present in the signal.<br></br>
<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*e-_z80BnbHWyFTfRLblJ_w.gif" width="400"><br></br>
`Fourier Transform` is a mathematical concept that can *decompose a signal into its constituent frequencies*. In its 2D plot output, the x-axis represent the signal frequencies and the y-axis represent their magnitudes (amplitude size). There is also `Inverse Fourier Transform` concept, which is just the opposite of the Fourier Transform.<br></br>
 For a better understanding of Fourier Transform output - let's create two simple sine waves, with two different frequencies:  *amplitude = 1 and frequency = 3* and *amplitude = 2 and frequency = 11*.
 Now, we can combine them into a single signal, that will look like that - <br></br>
<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*WBldOpArJgDXIFs5g_JydA.png" width="400"><br></br>
The output of Fourier Transform will show two spikes  for the two frequencies and their magnitudes <br></br>
<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*aIyR6XoUYGJp0_3Ug6iEyA.png" width="400"><br></br>
For more information, you can read the article - [understand autio fft](https://towardsdatascience.com/understanding-audio-data-fourier-transform-fft-spectrogram-and-speech-recognition-a4072d228520)

Having said that, we can use `Fourier Transform` to analyze frequencies of each spectrogram, it will include the following steps:
1. **Compute FFT for each image**: by using `np.fft.fft2` method
2. **Center frequencies**: move the zero frequency component to the center, by using `np.fft.fftshift`
3. **Extract magnitude spectrum for each image**: calculate the magnitude of the frequency components in a logarithmic scale by using `np.log1p`.
4. **Extract low and high frequencies for each image**: by using statistical thresholds, when can extract from a magnitude its low and high frequencies. 
>   - **Low Frequencies**: correspond to slower oscillations or longer wavelengths in the signal. Often, the fundamental frequencies of sounds or   signals fall in the low-frequency range and they have higher amplitude than highers ones.
located in the center of the magnitude spectrum and represent smooth variations and general trends in the image.
>   - **High Frequencies**:  correspond to faster oscillations or shorter wavelengths in the signal. In a spectrogram, these are represented by components that change rapidly over time.

Sounds like the basic "noising" signals - the vertical and the horizontal ones, will probably consider as **Low Frequencies**, and the needles will probably consider as **High Frequency**.

Let's start with extracting these features for each cadence snippet:

In [ ]:
def extract_magnitude_spectrum(spectrogram):
    spectrogram = spectrogram.astype(np.float32)
    spectrogram_frequencies_domain = np.fft.fft2(spectrogram)
    spectrogram_frequencies_domain = np.fft.fftshift(spectrogram_frequencies_domain)
    
    magnitude_spectrum = np.abs(spectrogram_frequencies_domain)
    # Apply logarithmic scaling and normalization for better visualization
    magnitude_spectrum_log = np.log1p(magnitude_spectrum)
    return (magnitude_spectrum_log - np.min(magnitude_spectrum_log)) / (np.max(magnitude_spectrum_log) - np.min(magnitude_spectrum_log))

def extract_cadence_frequency_features(cadence_snippet):
    magnitude_spectrums = []
    for spectrogram in cadence_snippet:
        magnitude_spectrums.append(extract_magnitude_spectrum(spectrogram))

    threshold = np.mean(magnitude_spectrums) + 2 * np.std(magnitude_spectrums)
    low_frequencies = np.sum(magnitude_spectrums <= threshold)
    high_frequencies = np.sum(magnitude_spectrums > threshold)

    return magnitude_spectrums, low_frequencies, high_frequencies

Now, we can use the code above for plotting these features for few samples from each Gaussian noise group (low, med and high):

In [ ]:
def plot_cadence_snippet_with_magnitudes(cadence_snippet, title, figsize=(10,4)):
    _, axes = plt.subplots(nrows=6, ncols=2, figsize=figsize)
    axes[0, 0].set_title('Original spectrogram', fontsize=10)
    axes[0, 1].set_title('Magnitude Spectrogram', fontsize=10)
    magnitude_spectrums, low_frequencies, high_frequencies = extract_cadence_frequency_features(cadence_snippet)
    
    print(title)
    print(f'Number of low frequencies: {low_frequencies}')
    print(f'Number of high frequencies: {high_frequencies}')

    for i in range(6):
        spectrogram = cadence_snippet[i].astype('float16')
        magnitude_spectrum = magnitude_spectrums[i].astype('float16')
        # For zoom-in
        center = np.array(magnitude_spectrum.shape) // 2
        crop_size = np.array(magnitude_spectrum.shape) // 2
        start = center - crop_size // 2
        end = center + crop_size // 2
        # For spectrogram channel text box
        channel_text = 'on' if i % 2 == 0 else 'off'
        channel_box = dict(facecolor='white', edgecolor='none', boxstyle='square,pad=0.3')

        ax_spectrogram = axes[i, 0]
        ax_spectrogram.imshow(spectrogram, aspect='auto', cmap='gray')
        ax_spectrogram.text(10, 90, channel_text, color='black', fontsize=8, va='center', ha='center', bbox=channel_box)
        ax_spectrogram.axis('off')
        
        ax_magnitude = axes[i, 1]
        ax_magnitude.imshow(magnitude_spectrum[start[0]:end[0], start[1]:end[1]], aspect='auto', cmap='inferno')
        ax_magnitude.text(7, 45, channel_text, color='black', fontsize=8, va='center', ha='center', bbox=channel_box)
        ax_magnitude.axis('off')

    plt.tight_layout()
    plt.show()

# Main
lowest_noise_image_0 = lowest_noise_images[lowest_noise_images['target'] == 0].head(1)['id'].iloc[0]
lowest_noise_image_1 = lowest_noise_images[lowest_noise_images['target'] == 1].head(1)['id'].iloc[0]
plot_cadence_snippet_with_magnitudes(np.load(get_file_path(lowest_noise_image_0, CFG.train_folder_path)), 'Low noise negative sample:')
plot_cadence_snippet_with_magnitudes(np.load(get_file_path(lowest_noise_image_1, CFG.train_folder_path)), 'Low noise positive sample:')

median_noise_image_0 = median_noise_images[median_noise_images['target'] == 0].head(1)['id'].iloc[0]
median_noise_image_1 = median_noise_images[median_noise_images['target'] == 1].head(1)['id'].iloc[0]
plot_cadence_snippet_with_magnitudes(np.load(get_file_path(median_noise_image_0, CFG.train_folder_path)), 'Med noise negative sample:')
plot_cadence_snippet_with_magnitudes(np.load(get_file_path(median_noise_image_1, CFG.train_folder_path)), 'Med noise positive sample:')

highest_noise_image_0 = highest_noise_images[highest_noise_images['target'] == 0].head(1)['id'].iloc[0]
highest_noise_image_1 = highest_noise_images[highest_noise_images['target'] == 1].head(1)['id'].iloc[0]
plot_cadence_snippet_with_magnitudes(np.load(get_file_path(highest_noise_image_0, CFG.train_folder_path)), 'High noise negative sample:')
plot_cadence_snippet_with_magnitudes(np.load(get_file_path(highest_noise_image_1, CFG.train_folder_path)), 'High noise positive sample:')

Interesting! We plotted for each one of our chosen data samples their magnitude spectrums and their high and low frequencies. As expected, from our plotted data samples it looks like the cleaner the image is - the more high frequencies we found.

Now, let's calculate these features on our entire training set so we will try to conculde some interesting points:

In [ ]:
def determine_noise_level(cadence_id):
    if cadence_id in lowest_noise_images['id'].values:
        return 'low'
    elif cadence_id in median_noise_images['id'].values:
        return 'medium'
    elif cadence_id in highest_noise_images['id'].values:
        return 'high'
    return 'medium'

# Main    
frequencies_list = []

for _, row in training_set.iterrows():
    file_id = row['id']
    file_path = get_file_path(file_id, CFG.train_folder_path)
    cadence_snippet = np.load(file_path)

    _, low_frequencies, high_frequencies = extract_cadence_frequency_features(cadence_snippet)

    frequencies = {
        'low_frequencies': low_frequencies,
        'high_frequencies': high_frequencies,
        'noise_level': determine_noise_level(file_id),
        'id': file_id,
        'target': row['target']
    }

    frequencies_list.append(frequencies)

frequencies_df = pd.DataFrame(frequencies_list)

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(15, 8))
axes = axes.flatten()

for index, noise_level in enumerate(['low', 'medium', 'high']):
    noise_df = frequencies_df[frequencies_df['noise_level'] == noise_level]
        
    ax_low_frequencies = axes[index * 2] 
    sns.histplot(data=noise_df, x='low_frequencies', hue='target', kde=True, ax=ax_low_frequencies)
    ax_low_frequencies.set_title(f'{noise_level.capitalize()} Noise - Low Frequencies by Target')
    ax_low_frequencies.set_xlabel('Low Frequency')
    ax_low_frequencies.set_ylabel('Count')
        
    ax_high_frequencies = axes[index * 2 + 1]  
    sns.histplot(data=noise_df, x='high_frequencies', hue='target', kde=True, ax=ax_high_frequencies)
    ax_high_frequencies.set_title(f'{noise_level.capitalize()} Noise - High Frequencies by Target')
    ax_high_frequencies.set_xlabel('High Frequency')
    ax_high_frequencies.set_ylabel('Count')
    
plt.tight_layout()
plt.show()

Great! It looks like the **Low Frequencies** amount is pretty much the same between the different noise level images, the main different is the amount of the **High Frequencies** - The images with the lowest noise has 12,000-18,000 high frequencies in average, the medium noise images has 5,000-15,000 high frequencies in average and the images with the highest noise has 1,000-3,000 high frequencies in average. We don't know yet if the small amount of high frequencies is due to bad images quality or another reason. Anyway, as we already said: **in the further sections - we might decide to consider some of the highest noise samples as outliers and remove them from our training set**.

Let's take a look over a `correlation matrix` as well, to examine if there is a correlation between freqency type to noise level:
> **Correlation Matrix**: Helps understand the relationships between the variables in a dataset based on their correlation coefficients. The diagonal values in the correlation matrix will always be 1, as a variable is perfectly correlated with itself - But, the off-diagonal elements show the correlation between different pairs of variables.

In [ ]:
noise_level_mapping = {'low': 0, 'medium': 1, 'high': 2}
frequencies_df['noise_level_numeric'] = frequencies_df['noise_level'].map(noise_level_mapping)

correlation_matrix = frequencies_df[['low_frequencies', 'high_frequencies', 'noise_level_numeric']].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.show()